### PROJECT - opening of new catering distributor centers in Minsk, Belarus

### Introduction/Business Problem

One distribution company decided to open its representative office in Minsk to sell various products for catering: kitchen and restaurant equipment, reusable and disposable tableware. In addition, it imports and sells premium coffee, tea and various snacks, etc. It is expected that about 5 warehouses will be opened in the city, from which goods will be delivered to the nearest points of sale. 

The objective of the study is to identify areas for each of the 5 points that will optimally correspond to the logistics tasks.

### Data

- Foursquare API - to get the venue data from location
- geolocator librarys - to update zip for venues that not represented in Foursquare
- Folium library - to plot the geo data
- sklearn - To do clustering on the data
- panda, nampu - python libraries to manupulate with data

All these services/libraries will be used to fetch and analyze existing foodservice points in Mins city where customer's firm can apply its business plans

In [ ]:
### all below steps/de

In [2]:
MINSK_LATITUDE = '53.893009'
MINSK_LONGITUDE = '27.567444'
print('The geograpical coordinates of MINSK are {}, {}.'.format(MINSK_LATITUDE, MINSK_LONGITUDE))

The geograpical coordinates of MINSK are 53.893009, 27.567444.


In [ ]:
!conda install -c conda-forge folium

In [11]:
import folium

minsk_map = folium.Map(location = [MINSK_LATITUDE, MINSK_LONGITUDE], zoom_start = 13)
folium.Marker([MINSK_LATITUDE, MINSK_LONGITUDE]).add_to(minsk_map)
minsk_map.save("Minsk Map.html")
minsk_map

In [82]:
CLIENT_ID = 'client_ID'
CLIENT_SECRET = 'client_secret'
RADIUS = 15000 # 4 Km
NO_OF_VENUES = 100
VERSION = '20180605' # '20190612'

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [83]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
import requests

pd.set_option('display.max_rows', None)

offset = 0
total_venues = 0
foursquare_venues = pd.DataFrame() # columns = ['name', 'categories', 'lat', 'lng']

while (True):
    url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
           '&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}').format(CLIENT_ID, 
                                                                        CLIENT_SECRET, 
                                                                        VERSION, 
                                                                        MINSK_LATITUDE, 
                                                                        MINSK_LONGITUDE, 
                                                                        RADIUS,
                                                                        NO_OF_VENUES,
                                                                        offset)
    result = requests.get(url).json()
    venues_fetched = len(result['response']['groups'][0]['items'])
    total_venues = total_venues + venues_fetched
    print("Total {} venues fetched within a total radius of {} Km".format(venues_fetched, RADIUS/1000))

    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)

#     print(venues.columns)
    # Filter the columns
#     filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    filtered_columns = [#'referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', #'venue.location.labeledLatLngs',
       'venue.location.distance',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.location.postalCode'
    ]
    venues = venues.loc[:, filtered_columns]

    # Filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

    # Clean all column names
#     venues.columns = [col.split(".")[-1] for col in venues.columns]
    foursquare_venues = pd.concat([foursquare_venues, venues], axis = 0, sort = False)
    
    if (venues_fetched < 100):
        break
    else:
        offset = offset + 100

foursquare_venues = foursquare_venues.reset_index(drop = True)
print("\nTotal {} venues fetched".format(total_venues))

Total 100 venues fetched within a total radius of 15.0 Km
Total 100 venues fetched within a total radius of 15.0 Km
Total 23 venues fetched within a total radius of 15.0 Km

Total 223 venues fetched


In [59]:
foursquare_venues

,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.formattedAddress,venue.categories,venue.location.postalCode
0,Moby Dick Gym,"Октябрьская ул., 16",53.891044,27.572110,"[{'label': 'display', 'lat': 53.89104378795624...",376,"[Октябрьская ул., 16, Мінск, Беларусь]",Gym / Fitness Center,220030
1,Yoga Place,"Октябрьская, 16",53.889890,27.574825,"[{'label': 'display', 'lat': 53.88988964556547...",595,"[Октябрьская, 16, Мінск, Беларусь]",Yoga Studio,220030
2,Burgerlab,ул. Октябрьская 19к4,53.891083,27.573041,"[{'label': 'display', 'lat': 53.89108258935017...",425,"[ул. Октябрьская 19к4, Мінск, Беларусь]",Burger Joint,220030
3,NEWTONLABS,"ул. Кирова, 19",53.896936,27.558829,"[{'label': 'display', 'lat': 53.89693612045103...",714,"[ул. Кирова, 19, Мінск, Беларусь]",IT Services,220030
4,Spa Beijing Hotel,Гостиница «Пекин»,53.892471,27.577848,"[{'label': 'display', 'lat': 53.89247070889114...",685,"[Гостиница «Пекин» (Красноармейская ул., 36), ...",Spa,220050
5,Silver Screen Cinemas,"Бобруйская ул., 6",53.890475,27.553810,"[{'label': 'display', 'lat': 53.89047492925527...",937,"[Бобруйская ул., 6, Мінск, 220030, Беларусь]",Multiplex,220030
6,Галерея современного искусства «Ў»,Октябрьская 19,53.891644,27.572143,"[{'label': 'display', 'lat': 53.89164423816109...",343,"[Октябрьская 19, Мінск, Беларусь]",Art Gallery,220030
7,Сон Гоголя,"ул. Ленина, 15",53.898806,27.561273,"[{'label': 'display', 'lat': 53.89880573392919...",761,"[ул. Ленина, 15, Мінск, Беларусь]",Bookstore,220030
8,Kew London Bar,проспект Независимости 18,53.899580,27.557199,"[{'label': 'display', 'lat': 53.89957981854969...",993,"[проспект Независимости 18, Мінск, Беларусь]",Bar,220030
9,Национальный художественный музей / National A...,"ул. Ленина, 20",53.898582,27.561276,"[{'label': 'display', 'lat': 53.89858181063965...",740,"[ул. Ленина, 20, Мінск, 220030, Беларусь]",Art Museum,220030


In [85]:
import geopy
import pandas as pd


geolocator = geopy.Nominatim(user_agent='my-application')


def update_zipcode(lat, lng, current_code):
    if pd.notna(current_code):
        return current_code
    else:
        location = geolocator.reverse((lat, lng))
        try:
            return location.raw['address']['postcode']
        except Exception as e:
            print(e)
            return current_code
            


foursquare_venues['venue.location.postalCode'] = foursquare_venues.apply(lambda x: update_zipcode(x['venue.location.lat'], x['venue.location.lng'], x['venue.location.postalCode']), axis=1)
foursquare_venues


'postcode'
'postcode'
'postcode'
'postcode'
'postcode'
'postcode'
'postcode'


,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.formattedAddress,venue.categories,venue.location.postalCode
0,Yoga Place,"Октябрьская, 16",53.889890,27.574825,"[{'label': 'display', 'lat': 53.88988964556547...",595,"[Октябрьская, 16, Мінск, Беларусь]",Yoga Studio,220030
1,Moby Dick Gym,"Октябрьская ул., 16",53.891044,27.572110,"[{'label': 'display', 'lat': 53.89104378795624...",376,"[Октябрьская ул., 16, Мінск, Беларусь]",Gym / Fitness Center,220030
2,NEWTONLABS,"ул. Кирова, 19",53.896936,27.558829,"[{'label': 'display', 'lat': 53.89693612045103...",714,"[ул. Кирова, 19, Мінск, Беларусь]",IT Services,220030
3,Silver Screen Cinemas,"Бобруйская ул., 6",53.890475,27.553810,"[{'label': 'display', 'lat': 53.89047492925527...",937,"[Бобруйская ул., 6, Мінск, 220030, Беларусь]",Multiplex,220030
4,Kew London Bar,проспект Независимости 18,53.899580,27.557199,"[{'label': 'display', 'lat': 53.89957981854969...",993,"[проспект Независимости 18, Мінск, Беларусь]",Bar,220030
5,Grand Café,"ул. Ленина, 2",53.902347,27.556641,"[{'label': 'display', 'lat': 53.90234735555793...",1258,"[ул. Ленина, 2 (Интернациональная ул.), Мінск,...",French Restaurant,220030
6,Spa Beijing Hotel,Гостиница «Пекин»,53.892471,27.577848,"[{'label': 'display', 'lat': 53.89247070889114...",685,"[Гостиница «Пекин» (Красноармейская ул., 36), ...",Spa,220050
7,Svobody 4,"пл. Свободы, 4",53.904034,27.554967,"[{'label': 'display', 'lat': 53.904034066868, ...",1475,"[пл. Свободы, 4, Мінск, Беларусь]",Wine Bar,220030
8,Burgerlab,ул. Октябрьская 19к4,53.891083,27.573041,"[{'label': 'display', 'lat': 53.89108258935017...",425,"[ул. Октябрьская 19к4, Мінск, Беларусь]",Burger Joint,220030
9,Vetka-Kvetka Flower Bar,"ул. Янки Купалы, 25",53.905385,27.561948,"[{'label': 'display', 'lat': 53.905385, 'lng':...",1424,"[ул. Янки Купалы, 25 (Интернациональная ул.), ...",Flower Shop,220030


In [86]:
foursquare_venues = foursquare_venues.dropna()
foursquare_venues.shape

(198, 9)

In [87]:
kclusters = foursquare_venues['venue.location.postalCode'].unique()
num_kclusters = len(kclusters)

In [88]:
# create map

map_clusters = folium.Map(location=[MINSK_LATITUDE, MINSK_LONGITUDE], zoom_start=12)

# set color scheme for the clusters
x = np.arange(num_kclusters)
ys = [i+x+(i*x)**2 for i in range(num_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, cluster in zip(foursquare_venues['venue.location.lat'], foursquare_venues['venue.location.lng'], foursquare_venues['venue.location.postalCode'], foursquare_venues['venue.location.postalCode']):
    label = folium.Popup('{} - Cluster {}'.format(post, cluster), parse_html=True)
    cluster = int(np.where(kclusters == cluster)[0])
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters.save("MINSK_ZIP_clusters.html")

In [89]:
def inline_map(m):
    from folium import Map
    from IPython.display import HTML, IFrame
    if isinstance(m, Map):
        m._build_map()
        srcdoc = m.HTML.replace('"', '&quot;')
        embed = HTML('<iframe srcdoc="{srcdoc}" '
                     'style="width: 100%; height: 500px; '
                     'border: none"></iframe>'.format(srcdoc=srcdoc))
    elif isinstance(m, str):
        embed = IFrame(m, width=1200, height=600)
    return embed
inline_map("MINSK_ZIP_clusters.html")

In [91]:
categories_unique = foursquare_venues['venue.categories'].unique()
categories_unique

array(['Yoga Studio', 'Gym / Fitness Center', 'IT Services', 'Multiplex',
       'Bar', 'French Restaurant', 'Spa', 'Wine Bar', 'Burger Joint',
       'Flower Shop', 'Hotel', 'Bookstore', 'Restaurant', 'Theater',
       'Arcade', 'Coffee Shop', 'Waterfront', 'Music Venue', 'Art Museum',
       'Bakery', 'Tennis Court', 'Cocktail Bar', 'Tea Room',
       'Art Gallery', 'Hostel', 'Café', 'Opera House', 'Sushi Restaurant',
       'Shoe Store', 'Gastropub', 'Whisky Bar', 'Park', 'Blini House',
       'Hookah Bar', 'Italian Restaurant', 'Dance Studio', 'Beach Bar',
       'Seafood Restaurant', 'Pub', 'Steakhouse', 'Beer Bar', 'Church',
       'Plaza', 'History Museum', 'Tapas Restaurant', 'Gym', 'Brewery',
       'Salon / Barbershop', 'Squash Court', 'Cupcake Shop', 'Aquarium',
       'Bistro', 'Auto Workshop', 'Dessert Shop', 'Japanese Restaurant',
       'Botanical Garden', 'Indian Restaurant', 'Lingerie Store',
       'Pelmeni House', 'Kebab Restaurant', 'Beer Store', 'Lounge',
       'C

In [94]:
categories_counts = foursquare_venues['venue.categories'].value_counts()
categories_counts

Coffee Shop               13
Gym / Fitness Center       9
Park                       9
Café                       8
Gym                        8
Bookstore                  7
Hotel                      6
Cocktail Bar               6
Spa                        6
Multiplex                  5
Arcade                     4
Beer Bar                   4
Wine Bar                   4
Restaurant                 4
Bakery                     4
Flower Shop                3
Beer Store                 3
Bistro                     3
Big Box Store              3
Tennis Court               3
Bar                        3
French Restaurant          3
Italian Restaurant         3
Waterfront                 2
Hookah Bar                 2
Dessert Shop               2
Lingerie Store             2
Theater                    2
Salad Place                2
Tea Room                   2
Sushi Restaurant           2
Tennis Stadium             2
Furniture / Home Store     2
Seafood Restaurant         2
Opera House   

In [105]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

foursquare_venues.head()

# # set number of clusters
kclusters = 5

venues_clustering = foursquare_venues.drop(["venue.name", "venue.categories", "venue.location.address","venue.location.labeledLatLngs", "venue.location.formattedAddress",
                                           "venue.location.distance", "venue.location.postalCode"], 1)

venues_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_clustering)

# # check cluster labels generated for each row in the dataframe
kmeans.labels_
venues_merged = venues_clustering.copy()

# # add clustering labels
venues_merged["Cluster"] = kmeans.labels_
venues_merged


,venue.location.lat,venue.location.lng,Cluster
0,53.889890,27.574825,2
1,53.891044,27.572110,2
2,53.896936,27.558829,2
3,53.890475,27.553810,2
4,53.899580,27.557199,2
5,53.902347,27.556641,2
6,53.892471,27.577848,2
7,53.904034,27.554967,2
8,53.891083,27.573041,2
9,53.905385,27.561948,2


In [107]:
# create map

map_clusters = folium.Map(location=[MINSK_LATITUDE, MINSK_LONGITUDE], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(venues_merged['venue.location.lat'], venues_merged['venue.location.lng'], venues_merged['Cluster']):
    label = folium.Popup('Cluster {}'.format(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters.save("MINSK_ZIP_by_clusters.html")

inline_map("MINSK_ZIP_by_clusters.html")